In [1]:
import keras
from keras.models import Sequential
# from keras.layers.core import Dense, Dropout, Activation, Flatten
# from keras.layers.convolutional import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.layers import Dropout, BatchNormalization, Input, Activation
from keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import *
from tensorflow.keras.utils import *

from keras.metrics import Recall
from keras.metrics import Precision
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.metrics import Recall
from keras.metrics import Precision
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [2]:
data = pd.read_csv("Desktop\dataset2020.csv")

In [3]:
mapping = {'pe-legit': 0, 'pe-malicious': 1}
data.iloc[:,0].replace(mapping, inplace=True)

In [4]:
data.fillna(0)

,pe-legit,0.0,0.0.1,0.0.2,0.0.3,1.0,0.0.4,0.0.5,0.0.6,1.0.1,...,0.0.167,0.0.168,0.0.169,0.0.170,0.0.171,0.976433728375,0.333333333333.1,0.333333333333.2,0.66719859844,0.116122928315
0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.412109,0.125000,0.125000,0.430123,0.017806
1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.270694,0.333333,0.666667,0.752563,0.003614
2,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.972070,0.125000,0.125000,0.613563,0.000037
3,1,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.989170,0.166667,0.166667,0.675976,0.000065
4,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.991901,0.500000,0.250000,0.734576,0.003825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49123,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.323838,1.000000,1.000000,0.979375,0.203325
49124,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.006316,1.000000,1.000000,1.000000,1.000000
49125,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.025753,0.200000,0.200000,0.434897,0.072456
49126,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.589556,0.250000,0.250000,0.607250,0.001549


In [5]:
y = np.array(data['pe-legit'])

In [6]:
y = to_categorical(y, num_classes=2)

In [7]:
x = np.array(data.drop('pe-legit', axis = 1))
# x = normalize(x)
x = np.reshape(x, (x.shape[0], x.shape[1], 1))


In [8]:
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 42, test_size = 0.2, shuffle = True)

In [9]:
import keras.backend as K
def f1_score(y_true, y_pred, threshold_shift=0):
    beta = 2

    y_pred = K.clip(y_pred, 0, 1)

    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))

In [17]:
n_timesteps = X_train.shape[1]
n_features  = X_train.shape[2] 
model = Sequential()

model.add(Conv1D(32, (5), activation='relu', input_shape=(n_timesteps, n_features)))  
model.add(MaxPooling1D(2,1))
model.add(Conv1D(64, (5), activation='relu'))  
model.add(MaxPooling1D(2,1))
model.add(Conv1D(128, (5), activation='relu'))  
model.add(MaxPooling1D(2,1))
model.add(Conv1D(128, (5), activation='relu'))  
model.add(MaxPooling1D(2,1))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1]))
model.add(Activation('softmax'))

In [18]:
OPTI = RMSprop(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer = OPTI, metrics = 
              ['accuracy',f1_score, Precision(name="precision"), Recall(name="recall")])

In [19]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 482, 32)           192       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 481, 32)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 477, 64)           10304     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 476, 64)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 472, 128)          41088     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 471, 128)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 471, 128)         

In [20]:
history = model.fit(X_train, y_train, batch_size=36, epochs=5, validation_split = 0.2, shuffle=True)

Train on 31441 samples, validate on 7861 samples
Epoch 1/5
31441/31441 [==============================] - 171s 5ms/step - loss: 0.3501 - accuracy: 0.8461 - f1_score: 0.8462 - precision: 0.8461 - recall: 0.8461 - val_loss: 0.2461 - val_accuracy: 0.9040 - val_f1_score: 0.9040 - val_precision: 0.9040 - val_recall: 0.9040
Epoch 2/5
31441/31441 [==============================] - 167s 5ms/step - loss: 0.2317 - accuracy: 0.9128 - f1_score: 0.9128 - precision: 0.9128 - recall: 0.9128 - val_loss: 0.2481 - val_accuracy: 0.9148 - val_f1_score: 0.9143 - val_precision: 0.9148 - val_recall: 0.9148
Epoch 3/5
31441/31441 [==============================] - 159s 5ms/step - loss: 0.1935 - accuracy: 0.9311 - f1_score: 0.9311 - precision: 0.9311 - recall: 0.9311 - val_loss: 0.2027 - val_accuracy: 0.9241 - val_f1_score: 0.9238 - val_precision: 0.9241 - val_recall: 0.9241
Epoch 4/5
31441/31441 [==============================] - 160s 5ms/step - loss: 0.1752 - accuracy: 0.9375 - f1_score: 0.9375 - precision: 0

In [21]:
f = model.evaluate(X_train, y_train)

39302/39302 [==============================] - 31s 798us/step


In [22]:
f

[0.13222646133088986,
 0.9541499018669128,
 0.9541802406311035,
 0.9541499018669128,
 0.9541499018669128]

In [26]:
n_timesteps1 = X_train.shape[1]
n_features1  = X_train.shape[2] 
model1 = Sequential()

model1.add(Conv1D(64, (5), activation='relu', input_shape=(n_timesteps, n_features)))  
model1.add(MaxPooling1D(2,1))
model1.add(Conv1D(128, (5), activation='relu'))  
model1.add(MaxPooling1D(2,1))
model1.add(Conv1D(256, (5), activation='relu'))  
model1.add(MaxPooling1D(2,1))
model1.add(Dropout(0.5))
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dense(y_train.shape[1]))
model1.add(Activation('softmax'))

In [27]:
import keras_metrics as km
OPTI = RMSprop(lr=0.001)
model1.compile(loss='binary_crossentropy', optimizer = OPTI, metrics = 
              ['accuracy', km.f1_score(), km.precision(), km.recall()])

tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fn
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> tp
tracking <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> fn


In [28]:
history = model1.fit(X_train, y_train, batch_size=36, epochs=5, validation_split = 0.2, shuffle=True)

Train on 31441 samples, validate on 7861 samples
Epoch 1/5
31441/31441 [==============================] - 381s 12ms/step - loss: 0.3612 - accuracy: 0.8449 - f1_score: 0.7860 - precision: 0.7782 - recall: 0.7949 - val_loss: 0.2603 - val_accuracy: 0.8966 - val_f1_score: 0.9037 - val_precision: 0.8537 - val_recall: 0.9600
Epoch 2/5
31441/31441 [==============================] - 369s 12ms/step - loss: 0.2349 - accuracy: 0.9110 - f1_score: 0.9079 - precision: 0.8884 - recall: 0.9282 - val_loss: 0.2311 - val_accuracy: 0.9148 - val_f1_score: 0.9142 - val_precision: 0.9249 - val_recall: 0.9039
Epoch 3/5
31441/31441 [==============================] - 373s 12ms/step - loss: 0.1910 - accuracy: 0.9334 - f1_score: 0.9330 - precision: 0.9111 - recall: 0.9561 - val_loss: 0.1798 - val_accuracy: 0.9345 - val_f1_score: 0.9377 - val_precision: 0.9139 - val_recall: 0.9628
Epoch 4/5
31441/31441 [==============================] - 368s 12ms/step - loss: 0.1760 - accuracy: 0.9406 - f1_score: 0.9399 - precisio

In [29]:
f = model1.evaluate(X_train, y_train)

39302/39302 [==============================] - 83s 2ms/step


In [31]:
f

[0.1341400176664326,
 0.9529286026954651,
 0.9563775062561035,
 0.9328163266181946,
 0.9811714887619019]